# Scaled Dot-Product Attention 계산 실습

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F 

# batch_size, seq_len, embedding_dim
x =  torch.tensor([[[1.0, 0.0, 1.0, 0.0],
                   [0.0, 2.0, 0.0, 2.0],
                   [1.0, 1.0, 1.0, 1.0]]]) 

 # 배치, 길이 차원 (1, 3, 4)

print("입력 x:", x.shape)



입력 x: torch.Size([1, 3, 4])


In [ ]:
# Q, K, V를 생성하는 선형층

W_q = nn.Linear(4, 4, bias=False)   # Query 생성용 선형 변환(4 -> 4)
W_k = nn.Linear(4, 4, bias=False)   # Key 생성용 선형 변환(4 -> 4)
W_v = nn.Linear(4, 4, bias=False)   # Value 생성용 선형 변환(4 -> 4)

# Q, K, V
Q = W_q(x)                           # x -> Q(배치, 길이 차원)
K = W_k(x)                           # x -> K
V = W_v(x)                           # x -> V


print("Q:" ,Q.shape)
print("K:" ,K.shape)
print("V:" ,V.shape)

Q: torch.Size([1, 3, 4])
K: torch.Size([1, 3, 4])
V: torch.Size([1, 3, 4])


In [ ]:
# 1. Q, K 유사도 계산
attn_scores = torch.matmul(Q, K.transpose(-2, -1))  # Q*K^T 토큰간 유사도(Score) 계산
attn_scores /= Q.size(-1) ** 0.5                    # 차원(d_k)로 나눠 Score 스케일 조정(Softmax 안정화)
print("attn_scores:", attn_scores.shape)            # score 행렬 (1, 3, 3) 

attn_scores: torch.Size([1, 3, 3])


In [ ]:
# 2. attention 분포 (확률)
attn_weights = F.softmax(attn_scores, dim=-1)        # 각 토큰이 바라볼 비율을 확률로 변환(행 단위 합 = 1)
print("attn_weights : ", attn_weights.shape)         # attention 가중치 (1, 3, 3)

attn_weights :  torch.Size([1, 3, 3])


In [ ]:
# 3. V-attention 분포의 가중합
output = torch.matmul(attn_weights, V)               # attention 가중치로 V를 가중합해 최종 출력 생성
print('attn_value :', output.shape)                  

attn_value : torch.Size([1, 3, 4])


In [7]:
# Attention 중간 결과(Q/K/V)와 분포, 최종 출력 확인
print("입력 x ", x)     # 원본 입력값
print("\nQ ", Q)        # Query 벡터(선형변환 결과)
print("\nK ", K)        # Key 벡터  (선형변환 결과)
print("\nV ", V)        # Value 벡터(선형변환 결과)

print("\nattention 분포 :", attn_weights)   # 가중치 : 각 토큰이 다른 토큰을 얼마나 참조하는지(확률 분포)
print("\n출력 output :", output)            # attention 가중합으로 만들어진 최종 출력 텐서

입력 x  tensor([[[1., 0., 1., 0.],
         [0., 2., 0., 2.],
         [1., 1., 1., 1.]]])

Q  tensor([[[-0.1392,  0.5523,  0.2820,  0.0902],
         [ 0.2970,  0.8269, -0.4085,  0.1327],
         [ 0.0093,  0.9658,  0.0777,  0.1565]]], grad_fn=<UnsafeViewBackward0>)

K  tensor([[[ 0.5735,  0.6561, -0.0016, -0.1080],
         [-0.7842, -0.0042, -0.3043, -0.1294],
         [ 0.1814,  0.6540, -0.1537, -0.1727]]], grad_fn=<UnsafeViewBackward0>)

V  tensor([[[ 0.1540,  0.6550, -0.3631,  0.0075],
         [ 0.2296,  1.0156, -1.2490,  0.8316],
         [ 0.2688,  1.1628, -0.9876,  0.4233]]], grad_fn=<UnsafeViewBackward0>)

attention 분포 : tensor([[[0.3473, 0.3045, 0.3481],
         [0.3804, 0.2514, 0.3683],
         [0.3705, 0.2641, 0.3654]]], grad_fn=<SoftmaxBackward0>)

출력 output : tensor([[[ 0.2170,  0.9416, -0.8503,  0.4032],
         [ 0.2153,  0.9327, -0.8158,  0.3677],
         [ 0.2159,  0.9358, -0.8253,  0.3770]]], grad_fn=<UnsafeViewBackward0>)


# Multi-Head-Attention (헤드 분할/결합) 계산

In [8]:
# batch_size, seq_len, embedding_dim
x = torch.tensor([[[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0],
                   [0.0, 2.0, 0.0, 2.0, 0.0, 2.0, 0.0, 2.0],
                   [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]]])  # 1, 3, 8

print("입력 x", x.shape)

입력 x torch.Size([1, 3, 8])


In [9]:
B, T, embedding_dim = x.shape           # B, T, E
num_head = 4                            # 헤드 개수
heading_dim = embedding_dim // num_head # 헤드당 차원 (d_k)

W_q = nn.Linear(embedding_dim, embedding_dim, bias=False)   # Query 생성용 선형 변환(8 -> 8)
W_k = nn.Linear(embedding_dim, embedding_dim, bias=False)   # Key   생성용 선형 변환
W_v = nn.Linear(embedding_dim, embedding_dim, bias=False)   # Value 생성용 선형 변환

# Q, K, V
Q = W_q(x)                           # (B, T, 8) -> (B, T, 8)
K = W_k(x)                           # (B, T, 8) -> (B, T, 8)
V = W_v(x)                           # (B, T, 8) -> (B, T, 8)


print("Q:" ,Q.shape)
print("K:" ,K.shape)
print("V:" ,V.shape)


Q: torch.Size([1, 3, 8])
K: torch.Size([1, 3, 8])
V: torch.Size([1, 3, 8])


In [ ]:
# 헤드 분할
# B, T, embedding_dim
# -> B, T, num_head, heading_dim
# -> B, num_head, T, heading_dim

Q_head = Q.view(B, T, num_head, heading_dim).transpose(1, 2)    # Q를 헤드별로 쪼개고(num_head) 차원 위치 교환
K_head = K.view(B, T, num_head, heading_dim).transpose(1, 2)    # K도 동일하게 헤드 분할
V_head = V.view(B, T, num_head, heading_dim).transpose(1, 2)    # V도 동일하게 헤드 분할

print("Q_head", Q_head.shape)   # (B, num_head, T, heading_dim)
print("K_head", K_head.shape)   # (B, num_head, T, heading_dim)
print("V_head", V_head.shape)   # (B, num_head, T, heading_dim)

Q_head torch.Size([1, 4, 3, 2])
K_head torch.Size([1, 4, 3, 2])
V_head torch.Size([1, 4, 3, 2])


In [11]:
# Q, K 유사도 계산
attn_scores = torch.matmul(Q_head, K_head.transpose(-2, -1))    # 각 헤드별로 Q*K^T 계산 (B, num_head, T, T)
attn_scores /= embedding_dim ** 0.5                             # 스케일링(defalut) 
print("attention_score :", attn_scores.shape)                   # score shape 출력

attention_score : torch.Size([1, 4, 3, 3])


In [12]:
# attention 분포 계산
attn_weights = F.softmax(attn_scores, dim=-1)   # 마지막 축을 기준으로 softmax -> 확률 분포
print("어텐션 분포 :", attn_weights.shape)       # (B, num_head, T, T)

어텐션 분포 : torch.Size([1, 4, 3, 3])


In [13]:
# V와 가중합 계산
output = torch.matmul(attn_weights, V_head)     # 가중합 -> (B, num_head, T, heading_dim)
print('출력 어텐션값 :', output.shape)           # 헤드별 출력 shape

출력 어텐션값 : torch.Size([1, 4, 3, 2])


In [ ]:
# 헤드 결합
output = output.transpose(1, 2)                         # (B, num_head, T, d_k) - > (B, T, num_head, d_k)
output = output.contiguous().view(B, T, embedding_dim)  # (B, T, num_head*d_k) -> (B, T, d_model)
print("출력 (헤드결합) : ", output.shape)

출력 (헤드결합) :  torch.Size([1, 3, 8])


tensor.contiguous() : view() 호출하기 전 메모리의 연속된 상태를 변환

- 일반 Attention vs Multi-Head Attention
(1) 같은 문장에서도 “관계”는 여러 종류라서  
예: “나는 어제 은행에 갔다”  
“은행”이 finance인지 river bank인지 문맥으로 판단해야 함  
어떤 헤드는 “시간/장소 단서”에  
다른 헤드는 “주변 단어 의미”에  
또 다른 헤드는 “문장 전역 정보”에 집중하는 식으로 동시에 여러 관계를 잡아냄  


(2) 긴 문장/복잡한 문맥에서 더 잘 버팀  
싱글 attention은 전역을 다 보긴 하지만 “한 가지 정렬”로만 보니까,  
복잡한 의존성이 많아질수록 한 번에 잡기 힘든데  
MHA는 여러 헤드가 분산해서 잡아주니 안정적.  

(3) 병렬 연산이 잘 맞아서(Transformer의 장점 극대화)  
RNN처럼 순차가 아니라 행렬곱 중심이라 GPU에서 효율이 좋고,  
MHA는 “여러 attention을 병렬로” 돌려도 구조적으로 잘 맞음.  